In [1]:
# !pip install map_boxes
# !pip install tqdm
# !pip install pycocotools

In [1]:
from map_boxes import mean_average_precision_for_boxes
import pandas as pd
import numpy as np
import json
from tqdm import tqdm
from pycocotools.coco import COCO

In [2]:
GT_JSON = '../dataset/train.json'
PRED_CSV = './work_dirs/inference/submission_epoch_48_train.csv'

# load ground truth
with open(GT_JSON, 'r') as outfile:
    test_anno = (json.load(outfile))

# load prediction
pred_df = pd.read_csv(PRED_CSV)

In [3]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2 confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''

new_pred = []

file_names = pred_df['image_id'].values.tolist()
bboxes = pred_df['PredictionString'].values.tolist()

'''
TODO create new_pred
'''
for i, bbox in enumerate(bboxes):
    if isinstance(bbox, float):
        print(f'{file_names[i]} empty box')

for file_name, bbox in tqdm(zip(file_names, bboxes)):
    boxes = np.array(str(bbox).split(' '))
    
    if len(boxes) % 6 == 1:
        boxes = boxes[:-1].reshape(-1, 6)
    elif len(boxes) % 6 == 0:
        boxes = boxes.reshape(-1, 6)
    else:
        raise Exception('error', 'invalid box count')
    for box in boxes:
        new_pred.append([file_name, box[0], box[1], float(box[2]), float(box[4]), float(box[3]), float(box[5])])

4883it [00:00, 16923.88it/s]


In [4]:
'''
[
    [file_name 1, confidence_score, x_min, x_max, y_min, y_max], 
    [file_name 2, confidence_score, x_min, x_max, y_min, y_max],
    ,,,
    [file_name , confidence_score, x_min, x_max, y_min, y_max]
]
'''
    
gt = []
    
   
'''
coco.getImgIds(): return image id list
    
coco.loadImgs(image_id): return image_info
    
image_info['file_name']: return file name
   
coco.getAnnIds(imgIds=image_info['id']): return annotation id
    
coco.loadAnns(ann_ids): return annotation information list (annotation_info_list)
    
annotation_info_list[i]['bbox']: return i'th annotation [x_min, y_min, w, h]
    
annotation_info_list[i]['category_id']: return i'th annotation category
    
'''
coco = COCO(GT_JSON)

'''
TODO create gt
'''
for image_id in coco.getImgIds():
        
    image_info = coco.loadImgs(image_id)[0]
    annotation_id = coco.getAnnIds(imgIds=image_info['id'])
    annotation_info_list = coco.loadAnns(annotation_id)
        
    file_name = image_info['file_name']
        
    for annotation in annotation_info_list:
        gt.append([file_name, annotation['category_id'],
                   float(annotation['bbox'][0]),
                   float(annotation['bbox'][0]) + float(annotation['bbox'][2]),
                   float(annotation['bbox'][1]),
                   (float(annotation['bbox'][1]) + float(annotation['bbox'][3]))])

loading annotations into memory...
Done (t=0.17s)
creating index...
index created!


In [5]:
num = 0
for line in gt:
    print(line)
    num += 1
    if num >= 10:
        break

['train/0000.jpg', 0, 197.6, 745.4, 193.7, 663.4]
['train/0001.jpg', 3, 0.0, 57.6, 407.4, 588.0]
['train/0001.jpg', 7, 0.0, 144.6, 455.6, 637.2]
['train/0001.jpg', 4, 722.3, 996.5999999999999, 313.4, 565.3]
['train/0001.jpg', 5, 353.2, 586.9, 671.0, 774.4]
['train/0001.jpg', 5, 3.7, 781.9000000000001, 448.5, 690.5]
['train/0001.jpg', 0, 425.3, 641.7, 681.9, 861.7]
['train/0001.jpg', 7, 92.4, 231.6, 601.7, 654.8000000000001]
['train/0001.jpg', 0, 622.4, 695.1999999999999, 686.5, 780.7]
['train/0002.jpg', 3, 267.9, 899.5, 165.2, 678.2]


In [6]:
num = 0
print(len(new_pred))
for line in new_pred:
    print(line)
    num += 1
    if num >= 30:
        break

50778
['train/0000.jpg', '0', '0.9999628', 198.51425, 766.6421, 209.66875, 660.0632]
['train/0001.jpg', '0', '0.947628', 91.74872, 227.90884, 596.2254, 655.86017]
['train/0001.jpg', '0', '0.9416905', 430.87857, 631.49805, 685.6425, 856.11786]
['train/0001.jpg', '0', '0.26931238', 5.8422546, 134.04042, 463.8631, 613.9615]
['train/0001.jpg', '0', '0.22473072', 2.1472187, 60.9804, 384.6396, 593.5803]
['train/0001.jpg', '0', '0.15670538', 610.7492, 675.5395, 696.95667, 797.91284]
['train/0001.jpg', '0', '0.05740051', 53.644672, 144.05804, 455.85324, 510.4683]
['train/0001.jpg', '1', '0.1272443', 26.08231, 140.52888, 460.0509, 598.1904]
['train/0001.jpg', '1', '0.08180946', 18.94104, 786.7562, 453.05133, 695.7972]
['train/0001.jpg', '3', '0.09116503', 18.94104, 786.7562, 453.05133, 695.7972]
['train/0001.jpg', '3', '0.058742765', 1.3517132, 47.58043, 381.01163, 609.19763]
['train/0001.jpg', '4', '0.99884415', 721.458, 991.3627, 312.6988, 571.58923]
['train/0001.jpg', '4', '0.13023937', 6.01

In [7]:
'''
calculate mAP
'''

'''
Special mission you sould create mean_average_precision_for_boxes function, not library
'''

mean_ap, average_precisions = mean_average_precision_for_boxes(gt, new_pred, iou_threshold=0.5)

print(mean_ap)

Number of files in annotations: 4883
Number of files in predictions: 4883
Unique classes: 10
Detections length: 4883
Annotations length: 4883
0                              | 0.654332 |    3966
1                              | 0.685738 |    6352
2                              | 0.827648 |     897
3                              | 0.796489 |     936
4                              | 0.828888 |     982
5                              | 0.728368 |    2943
6                              | 0.740184 |    1263
7                              | 0.840874 |    5178
8                              | 0.927311 |     159
9                              | 0.885422 |     468
mAP: 0.791525
0.7915254396585354
